In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1, )
examples = [
    {
        "movie": "가타카",
        "answer": """
        - 감독:앤드류니콜,
        - 출연:에단 호크, 주드 로, 우마 서먼
        - 예산:$3600만,
        - 흥행 수익: $12,532,777,
        - 영화의 장르: SF, 드라마, 스릴러,
        - 시놉시스: 빈센트는 자연의 섭리에 의해 태어난 인간이다. 반면 그의 동생 안톤은 유전자의 선택으로 태어났다. 빈센트는 약하고 병에 잘 걸리며 유전자적 열성인 근시이다. 빈센트의 꿈은 우주 비행이지만 그러한 약체 때문에 우주탐사팀을 보내는 회사인 가타카에서 청소부로 밖에 취직할 수 없다. 그러던 중 빈센트는 DNA 중개인을 통해 교통사고로 불구가 된 수영선수이자 유전학적으로 우성인 제롬 모로우를 소개받고 그의 유전인자를 돈으로 사게 된다. 그리하여 제롬의 유전인자로 가타카에 엘리트 사원으로 취직한 빈센트는 거기서 미모의 여성 아일린과 사랑에 빠지는데...

        """,
    },
    {
        "movie": "인터스텔라",
        "answer": """
        - 감독: 크리스토퍼 놀란
        - 출연: 매튜 맥커너히, 앤 해서웨이, 제시카 차스테인, 마이클 케인
        - 예산: $1억 6500만
        - 흥행 수익: 약 $6억 7700만
        - 장르: SF, 드라마, 어드벤처
        - 시놉시스: 가까운 미래, 지구의 식량 위기를 해결하기 위해 나선 전직 NASA 파일럿 쿠퍼는 웜홀을 통해 다른 은하계를 탐사하는 극비 프로젝트에 참여한다. 그곳에서 그는 인류의 새로운 터전을 찾기 위해 시간과 중력, 사랑의 경계를 넘나드는 여정에 오른다.
        """,
    },
    {
        "movie": "라라랜드",
        "answer": """
        - 감독: 데이미언 셔젤
        - 출연: 라이언 고슬링, 엠마 스톤
        - 예산: $3000만
        - 흥행 수익: 약 $4억 4600만
        - 장르: 뮤지컬, 드라마, 로맨스
        - 시놉시스: 꿈을 꾸는 이들의 도시 LA에서, 재즈 피아니스트 세바스찬과 우연히 만난 배우 지망생 미아는 서로의 꿈과 사랑에 대해 공감하며 서로에게 큰 영감을 준다. 하지만 그들의 꿈이 현실이 되면서 둘 사이에는 예상치 못한 갈등이 생기기 시작한다.
        """,
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [("human", "영화 {movie}에 대해 알려줘"), ("ai", "{answer}")]
)

movie_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "너는 세계의 영화 정보를 모두 알고 있어"),
        movie_prompt,
        ("human", "{movie}")
    ]
)

chain = final_prompt | chat
chain.invoke({"movie": "다크나이트"})


AIMessage(content='\n        - 감독: 크리스토퍼 놀란\n        - 출연: 크리스찬 베일, 히스 레저, 앤 해서웨이, 마이클 케인\n        - 예산: $1억 8500만\n        - 흥행 수익: 약 $10억 4800만\n        - 장르: 액션, 스릴러, 범죄\n        - 시놉시스: 배트맨이라는 영웅으로 변신한 브루스 웨인은 고담시를 위협하는 범죄와 악당들과 맞서 싸운다. 특히 조커라는 미친 범죄자의 등장으로 도시는 큰 위기에 놓이게 되고, 배트맨은 조커의 악랄한 계획을 막기 위해 힘을 모은다. 이 과정에서 배트맨과 조커의 대결은 도시와 주변 사람들에게 큰 영향을 미치게 된다.')